# Workshop 2 DataScrapping 
Students:
    Edgardo Ortiz
    Nicolas Henriquez
    Carlo 
    Benjamin



## 1.1 Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import numpy as np
from tqdm import tqdm  # Importa tqdm
from selenium.webdriver.firefox.options import Options as FirefoxOptions

## 1.2.1 Hockey Team Scrapping

In [2]:
hockey_url = 'https://www.scrapethissite.com/pages/forms/?page_num=1'

# Web scraping
driver = webdriver.Firefox()
driver.get(hockey_url)
data = []

while True:
    table = driver.find_element(By.TAG_NAME, "table")
    table_html = table.get_attribute("outerHTML")
    soup = BeautifulSoup(table_html, "html.parser")

    rows = soup.find_all("tr")

    for row in rows:
        cells = row.find_all("td")  # Change "td" to "th" if you also want table headers
        row_data = [cell.get_text() for cell in cells]
        data.append(row_data)
    
    try:
        next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next']")
    except NoSuchElementException:
        break

    next_button.click()

driver.close()

## 1.2.2 Hockey Data Cleanning

In [4]:
        # Create a pandas DataFrame with the scraped data
hockey_df = pd.DataFrame(data)

# Drop the first row (index 0) of the DataFrame
hockey_df = hockey_df.iloc[1:]

# Set the headers
new_headers = ["Team Name", "Year", "Wins", "Losses", "OT Losses", "Win %", "Goals For (GF)", "Goals Against (GA)", "plusMinus"]
hockey_df.columns = new_headers

# Replace '\n' with an empty string '' in all cells of the DataFrame
hockey_df = hockey_df.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)

numeric_columns = ["Year", "Wins", "Losses", "OT Losses", "Win %", "Goals For (GF)", "Goals Against (GA)", "plusMinus"]

# Convert numeric columns to numeric data types
hockey_df[numeric_columns] = hockey_df[numeric_columns].apply(pd.to_numeric, errors="coerce")

# Drop rows that are completely empty
hockey_df = hockey_df.dropna(how='all')

# Replace NaN with 0 in the "OT Losses" column
hockey_df["OT Losses"].fillna(0, inplace=True)

# Remove leading and trailing white spaces from the "Team Name" column
hockey_df["Team Name"] = hockey_df["Team Name"].str.strip()

#recalculate winrate
hockey_df['Win %'] = hockey_df['Wins'] / (hockey_df['Wins'] + hockey_df['Losses'] + hockey_df['OT Losses']) * 100

# Save the cleaned DataFrame to an Excel file
hockey_df.to_excel("hockey_cleaned.xlsx", index=False)

# Display the cleaned DataFrame
hockey_df

C:\Users\nicol\AppData\Local\Temp\ipykernel_19556\3757121240.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  hockey_df = hockey_df.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)


,Team Name,Year,Wins,Losses,OT Losses,Win %,Goals For (GF),Goals Against (GA),plusMinus
1,Boston Bruins,1990.0,44.0,24.0,0.0,64.705882,299.0,264.0,35.0
2,Buffalo Sabres,1990.0,31.0,30.0,0.0,50.819672,292.0,278.0,14.0
3,Calgary Flames,1990.0,46.0,26.0,0.0,63.888889,344.0,263.0,81.0
4,Chicago Blackhawks,1990.0,49.0,23.0,0.0,68.055556,284.0,211.0,73.0
5,Detroit Red Wings,1990.0,34.0,38.0,0.0,47.222222,273.0,298.0,-25.0
...,...,...,...,...,...,...,...,...,...
601,Tampa Bay Lightning,2011.0,38.0,36.0,8.0,46.341463,235.0,281.0,-46.0
602,Toronto Maple Leafs,2011.0,35.0,37.0,10.0,42.682927,231.0,264.0,-33.0
603,Vancouver Canucks,2011.0,51.0,22.0,9.0,62.195122,249.0,198.0,51.0
604,Washington Capitals,2011.0,42.0,32.0,8.0,51.219512,222.0,230.0,-8.0


## 1.3.1 Oscar Winners Scrapping

Meterse a wikipedia, obtener desde 1980 y obtener director y cuanto costo y recaudo



In [ ]:
driver = webdriver.Firefox()
oscar_url = 'https://www.scrapethissite.com/pages/ajax-javascript/'
driver.get(oscar_url)

year_buttons = driver.find_elements(By.CLASS_NAME, "year-link")

data = []

for button in year_buttons:
    time.sleep(5)
    year_id = button.get_attribute("id")
    year_text = button.text
    print(year_text)

    # Hacer clic en el botón para obtener más datos si es necesario
    button.click()

    table = driver.find_element(By.TAG_NAME, "table")
    table_html = table.get_attribute("outerHTML")
    soup = BeautifulSoup(table_html, "html.parser")

    rows = soup.find_all("tr")

    for row in rows:
        cells = row.find_all("td")  # Change "td" to "th" if you also want table headers
        row_data = [year_text]

        for cell in cells:

            if "film-best-picture" in cell.get("class", []):
                # Busca el ícono "glyphicon-flag" dentro de la celda
                flag_icon = cell.find("i", class_="glyphicon glyphicon-flag")
                # Verifica si se encontró el ícono y agrega "True" o "False" en consecuencia
                best_picture = "True" if flag_icon else "False"
                row_data.append(best_picture)
            else:
                row_data.append(cell.get_text())

        data.append(row_data)

driver.quit()

## 1.3.2 Awards Cleanning

In [ ]:
# Crear un DataFrame de pandas con los datos
column_names = ["Year", "Title", "Nominations", "Awards", "Best Picture"]
df_awards = pd.DataFrame(data, columns=column_names)
# Convertir "Nominations" y "Awards" a números
df_awards["Nominations"] = pd.to_numeric(df_awards["Nominations"], errors="coerce")
df_awards["Awards"] = pd.to_numeric(df_awards["Awards"], errors="coerce")
# Convertir "Año" a un número
df_awards["Year"] = pd.to_numeric(df_awards["Year"], errors="coerce")
df_awards = df_awards.dropna(thresh=3)
df_awards.to_excel("awards_cleaned.xlsx", index=False)
df_awards

## 1.4.1 Turtles Scrapping

In [ ]:
driver = webdriver.Firefox()

turtle_url = 'https://www.scrapethissite.com/pages/frames/'
driver.get(turtle_url)

iframe = driver.find_element(By.ID, 'iframe')
driver.switch_to.frame(iframe)

learn_more_buttons = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'btn.btn-default.btn-xs'))
)

href_values = []

for button in learn_more_buttons:
    href_value = button.get_attribute("href")
    if href_value:
        href_values.append(href_value)

lead_list = []
image_src_list = []

for href in href_values:
    driver.get(href)
    time.sleep(2)

    image_element = driver.find_element(By.CLASS_NAME, 'turtle-image.center-block')
    image_src = image_element.get_attribute("src")
    image_src_list.append(image_src)

    lead_element = driver.find_element(By.CLASS_NAME, 'lead')
    lead_text = lead_element.text
    lead_list.append(lead_text)

driver.switch_to.default_content()
driver.quit()


## 1.4.2 Turtles transformation

In [ ]:
# Regex
regex = r'The (.*?) family of turtles — more commonly known as "(.*?)" — were first discovered in (\d{4}) by ([^.]+)\.'

# List to store information
turtle_info = []

# Iterate through each lead element
for lead_text in lead_list:
    # Apply the regex to extract information
    result = re.search(regex, lead_text)
    
    # Check if the pattern was found
    if result:
        name = result.group(1)
        common_name = result.group(2)
        discovery_year = result.group(3)
        discovered_by = result.group(4)
    else:
        name = "Information not found"
        common_name = "Information not found"
        discovery_year = "Information not found"
        discovered_by = "Information not found"
    
    # Add the information to the list
    turtle_info.append({
        'name': name,
        'common_name': common_name,
        'discovery_year': discovery_year,
        'discovered_by': discovered_by
    })

turtle_df = pd.DataFrame(turtle_info)

turtle_df['image_url'] = image_src_list

# Show the DataFrame with the new column
turtle_df.to_excel("turtle_cleaned.xlsx", index=False)
turtle_df


# 2 Secret Tasks


### 2.1 Secret Task Awards

In [4]:
options = FirefoxOptions()
options.add_argument("--headless")
# Inicializa el controlador de Firefox
driver = webdriver.Firefox(options=options)

awards_secret_url = 'https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
driver.get(awards_secret_url)
time.sleep(5)

data = []

table = driver.find_element(By.TAG_NAME, "table")
table_html = table.get_attribute("outerHTML")
soup = BeautifulSoup(table_html, "html.parser")

# Encuentra todas las filas de la tabla (supongo que están en una etiqueta 'tr')
rows = soup.find_all('tr')

for row in rows:
    if 'style' in row.attrs and 'background' in row['style']:
        has_background = True
    else:
        has_background = False

    columns = row.find_all('td')
    if len(columns) >= 4:
        # Inicializa variables con valores predeterminados
        nombre = ''
        href = ''
        año = ''
        premios = '0'
        nominaciones = '0'

        # Intenta extraer los datos si están disponibles
        if columns[0].find('i'):
            nombre = columns[0].find('i').text.strip()
            if columns[0].find('a'):
                href = columns[0].find('a')['href']
        if columns[1].text.strip():
            año = columns[1].text.strip()
        if columns[2].text.strip():
            premios = columns[2].text.strip()
        if columns[3].text.strip():
            nominaciones = columns[3].text.strip()

        # Imprime si la fila tiene fondo o no
        if has_background:
            print("Esta fila tiene fondo:")
        else:
            print("Esta fila no tiene fondo:")

        # Imprime los datos que has extraído
        print(f'Nombre: {nombre}')
        print(f'Año: {año}')
        print(f'Premios: {premios}')
        print(f'Nominaciones: {nominaciones}')
        if href:
            link = f'https://en.wikipedia.org{href}'
            print(f'Href: {link}')
        else:
            link = ''
            print('No se encontró un enlace.')

        print()
        

        # Agrega los datos a la lista data
        data.append({
            'Title': nombre,
            'Year': año,
            'Awards': premios,
            'Nominations': nominaciones,
            'IsBestpicture': 1 if has_background else 0,
            'Link': link
        })

# Cierra el controlador de Firefox
driver.close()

Esta fila tiene fondo:
Nombre: Everything Everywhere All at Once
Año: 2022
Premios: 7
Nominaciones: 11
Href: https://en.wikipedia.org/wiki/Everything_Everywhere_All_at_Once

Esta fila no tiene fondo:
Nombre: All Quiet on the Western Front
Año: 2022
Premios: 4
Nominaciones: 9
Href: https://en.wikipedia.org/wiki/All_Quiet_on_the_Western_Front_(2022_film)

Esta fila no tiene fondo:
Nombre: The Whale
Año: 2022
Premios: 2
Nominaciones: 3
Href: https://en.wikipedia.org/wiki/The_Whale_(2022_film)

Esta fila no tiene fondo:
Nombre: Top Gun: Maverick
Año: 2022
Premios: 1
Nominaciones: 6
Href: https://en.wikipedia.org/wiki/Top_Gun:_Maverick

Esta fila no tiene fondo:
Nombre: Black Panther: Wakanda Forever
Año: 2022
Premios: 1
Nominaciones: 5
Href: https://en.wikipedia.org/wiki/Black_Panther:_Wakanda_Forever

Esta fila no tiene fondo:
Nombre: Avatar: The Way of Water
Año: 2022
Premios: 1
Nominaciones: 4
Href: https://en.wikipedia.org/wiki/Avatar:_The_Way_of_Water

Esta fila no tiene fondo:
Nombre

In [5]:
awards_secret_df = pd.DataFrame(data)
awards_secret_df = awards_secret_df[awards_secret_df['Year'] >= '1980']
awards_secret_df

,Title,Year,Awards,Nominations,IsBestpicture,Link
0,Everything Everywhere All at Once,2022,7,11,1,https://en.wikipedia.org/wiki/Everything_Every...
1,All Quiet on the Western Front,2022,4,9,0,https://en.wikipedia.org/wiki/All_Quiet_on_the...
2,The Whale,2022,2,3,0,https://en.wikipedia.org/wiki/The_Whale_(2022_...
3,Top Gun: Maverick,2022,1,6,0,https://en.wikipedia.org/wiki/Top_Gun:_Maverick
4,Black Panther: Wakanda Forever,2022,1,5,0,https://en.wikipedia.org/wiki/Black_Panther:_W...
5,Avatar: The Way of Water,2022,1,4,0,https://en.wikipedia.org/wiki/Avatar:_The_Way_...
6,Women Talking,2022,1,2,0,https://en.wikipedia.org/wiki/Women_Talking_(f...
7,Guillermo del Toro's Pinocchio,2022,1,1,0,https://en.wikipedia.org/wiki/Guillermo_del_To...
8,Navalny,2022,1,1,0,https://en.wikipedia.org/wiki/Navalny_(film)
9,The Elephant Whisperers,2022,1,1,0,https://en.wikipedia.org/wiki/The_Elephant_Whi...


In [6]:
def scrape_website(link):
    if not link:
        print("El enlace está vacío.")
        return np.nan, np.nan

    driver = webdriver.Firefox(options=options)
    driver.get(link)
    time.sleep(3) 

    budget = None
    box_office = None
    director = None

    try:
        budget_element = driver.find_element(By.XPATH, "//th[contains(text(),'Budget')]/following-sibling::td[@class='infobox-data']")
        budget = budget_element.text.strip()
        
    except Exception as e:
        print(f"No se encontró el elemento 'Budget'. Error: {e}")
        print(link)

    try:
        # Intenta encontrar el elemento de la taquilla
        box_office_element = driver.find_element(By.XPATH, "//th[contains(text(),'Box office')]/following-sibling::td[@class='infobox-data']")
        box_office = box_office_element.text.strip()
    except Exception as e:
        print(f"No se encontró el elemento 'Box office'. Error: {e}")
        print(link)
    try:
        # Busca la etiqueta <th> que contiene 'Directed by'
        director_element = driver.find_element(By.XPATH, "//th[contains(text(),'Directed by')]/following-sibling::td[@class='infobox-data']")
        # Extrae el texto del nombre del director
        director = director_element.text.strip()
    except Exception as e:
        print(f"No se encontró el elemento 'Directed by'. Error: {e}")
        director = None  # Otra opción es asignar None si no se encuentra el nombre del director


    driver.quit()  

    budget = parse_budget(budget)
    box_office = parse_box_office(box_office)
    director = parse_director(director)

    return budget, box_office, director

def parse_director(director):
    if not director:
        return None 
    else:
    
        return director.strip() 


def parse_budget(budget):
    if not budget:
        return 0
    try:
        
        budget = re.sub(r'\[[0-9]+\]', '', budget) 
        matches = re.findall(r'([\d.]+)', budget) 
        if matches:
           
            values = [float(match) for match in matches]
            return min(values)
        else:
            return 0
    except ValueError:
        return 0

def parse_box_office(box_office):
    if not box_office:
        return 0
    try:
        box_office = re.sub(r'\[[0-9]+\]', '', box_office)  
        matches = re.findall(r'([\d.]+)', box_office)  
        if matches:
            # Convierte los valores a flotantes y toma el valor más bajo
            values = [float(match) for match in matches]
            return min(values)
        else:
            return 0
    except ValueError:
        return 0


In [ ]:
#scrape_website("https://en.wikipedia.org/wiki/All_Quiet_on_the_Western_Front_(2022_film)")

In [7]:
revenues = []
box_offices = []
directors = []

# Itera a través de las filas del DataFrame
for index, row in tqdm(awards_secret_df.iterrows(), total=len(awards_secret_df)):
    link = row['Link']
    revenue, box_office, director = scrape_website(link)
    revenues.append(revenue)
    box_offices.append(box_office)
    directors.append(director)

# Crea tres Series a partir de las listas de valores
revenue_series = pd.Series(revenues, name='Revenue')
box_office_series = pd.Series(box_offices, name='Box Office')
director_series = pd.Series(directors, name='Director')

# Agrega las Series al DataFrame original
awards_secret_df['Revenue'] = revenue_series
awards_secret_df['Box Office'] = box_office_series
awards_secret_df['Director'] = director_series 



  4%|▎         | 1/28 [00:15<07:11, 15.99s/it]

No se encontró el elemento 'Box office'. Error: Message: Unable to locate element: //th[contains(text(),'Box office')]/following-sibling::td[@class='infobox-data']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

https://en.wikipedia.org/wiki/All_Quiet_on_the_Western_Front_(2022_film)


 21%|██▏       | 6/28 [01:36<05:53, 16.06s/it]

No se encontró el elemento 'Budget'. Error: Message: Unable to locate element: //th[contains(text(),'Budget')]/following-sibling::td[@class='infobox-data']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

https://en.wikipedia.org/wiki/Women_Talking_(film)


 29%|██▊       | 8/28 [02:07<05:20, 16.00s/it]

...

 68%|██████▊   | 19/28 [05:02<02:22, 15.85s/it]

No se encontró el elemento 'Budget'. Error: Message: Unable to locate element: //th[contains(text(),'Budget')]/following-sibling::td[@class='infobox-data']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

https://en.wikipedia.org/wiki/The_Queen_of_Basketball
No se encontró el elemento 'Box office'. Error: Message: Unable to locate element: //th[contains(text(),'Box office')]/following-sibling::td[@class='infobox-data']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://

 71%|███████▏  | 20/28 [05:18<02:06, 15.85s/it]

No se encontró el elemento 'Budget'. Error: Message: Unable to locate element: //th[contains(text(),'Budget')]/following-sibling::td[@class='infobox-data']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

https://en.wikipedia.org/wiki/Summer_of_Soul


100%|██████████| 28/28 [07:25<00:00, 15.91s/it]


In [8]:
def limpiar_valor(valor):
    valor_limpio = re.sub(r'[^0-9]', '', valor)
    return valor_limpio

awards_secret_df["Awards"] = awards_secret_df["Awards"].apply(limpiar_valor)
awards_secret_df["Nominations"] = awards_secret_df["Nominations"].apply(limpiar_valor)

def limpiar_anio(anio):
    if isinstance(anio, str):
        anio_limpio = anio.split('/')[0]  # Tomar solo los primeros 4 caracteres
        return anio_limpio.strip()  # Eliminar espacios en blanco
    return anio

awards_secret_df["Year"] = awards_secret_df["Year"].apply(limpiar_anio)

def parse_director(director):
    if not director:
        return None  # Si el nombre del director es nulo o vacío, devolvemos None
    
    # Dividir el nombre del director en múltiples nombres si hay saltos de línea (\n)
    director_names = [name.strip() for name in director.split('\n')]
    
    # Formatear los nombres como "director1, director2, ..."
    formatted_director = ', '.join(director_names)
    
    return formatted_director

awards_secret_df["Director"] = awards_secret_df["Director"].apply(parse_director)

# Save the cleaned DataFrame to an Excel file
awards_secret_df.to_excel("awards_secret.xlsx", index=False)
awards_secret_df

,Title,Year,Awards,Nominations,IsBestpicture,Link,Revenue,Box Office,Director
0,Everything Everywhere All at Once,2022,7,11,1,https://en.wikipedia.org/wiki/Everything_Every...,14.3,141.200,"Daniel Kwan, Daniel Scheinert"
1,All Quiet on the Western Front,2022,4,9,0,https://en.wikipedia.org/wiki/All_Quiet_on_the...,20.0,0.000,Edward Berger
2,The Whale,2022,2,3,0,https://en.wikipedia.org/wiki/The_Whale_(2022_...,3.0,54.900,Darren Aronofsky
3,Top Gun: Maverick,2022,1,6,0,https://en.wikipedia.org/wiki/Top_Gun:_Maverick,170.0,1.496,Joseph Kosinski
4,Black Panther: Wakanda Forever,2022,1,5,0,https://en.wikipedia.org/wiki/Black_Panther:_W...,200.0,859.200,Ryan Coogler
5,Avatar: The Way of Water,2022,1,4,0,https://en.wikipedia.org/wiki/Avatar:_The_Way_...,350.0,2.320,James Cameron
6,Women Talking,2022,1,2,0,https://en.wikipedia.org/wiki/Women_Talking_(f...,0.0,9.000,Sarah Polley
7,Guillermo del Toro's Pinocchio,2022,1,1,0,https://en.wikipedia.org/wiki/Guillermo_del_To...,35.0,109.000,"Guillermo del Toro, Mark Gustafson"
8,Navalny,2022,1,1,0,https://en.wikipedia.org/wiki/Navalny_(film),0.0,106.000,Daniel Roher
9,The Elephant Whisperers,2022,1,1,0,https://en.wikipedia.org/wiki/The_Elephant_Whi...,0.0,0.000,Kartiki Gonsalves


### 2.2 Turtle Secret


In [ ]:

def split_family_author(text):
    match = re.match(r'([A-Z][a-z]*)([A-Z].*)', text)
    if match:
        family_name = match.group(1)
        author_name = match.group(2)
        return family_name, author_name
    else:
        return text, ""

def extract_family_author_year(text):
    match = re.match(r'([A-Z][a-z]*)([A-Z].*), (\d{4})', text)
    if match:
        family = match.group(1)
        author = match.group(2)
        year = match.group(3)
        return family, author, year
    else:
        return "", "", ""

In [ ]:
text = "CarettochelyidaeBoulenger, 1887"
family, author, year = extract_family_author_year(text)
print("Family:", family)
print("Author:", author)
print("Year:", year)

In [ ]:
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium import webdriver

# Configurar el navegador
options = FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

awards_secret_url = 'https://en.wikipedia.org/wiki/List_of_Testudines_families'
driver.get(awards_secret_url)

table = driver.find_element(By.TAG_NAME, "table")
table_html = table.get_attribute("outerHTML")
soup = BeautifulSoup(table_html, "html.parser")

dfs_to_concat = []

rows = soup.find_all("tr")
for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 5:
        family_author_text = cells[0].get_text(strip=True)
        family, author, year = extract_family_author_year(family_author_text)
        genera = cells[1].get_text(strip=True)
        common_name = cells[2].get_text(strip=True)
        example_species = cells[3].get_text(strip=True)
        df = pd.DataFrame({"Family": [family], "Genera": [genera], "Common Name": [common_name], "Example Species": [example_species], "Author": [author], "Year": [year]})
        dfs_to_concat.append(df)

driver.close()

turtle_secret_df = pd.concat(dfs_to_concat, ignore_index=True)

turtle_secret_df

In [ ]:
turtle_secret_df['URL'] = 'https://en.wikipedia.org/wiki/' + turtle_secret_df['Family']

turtle_secret_df

In [ ]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
import pandas as pd

# Configurar el navegador
options = FirefoxOptions()
options.add_argument("--headless")
driver = Firefox(options=options)

for index, row in turtle_secret_df.iterrows():
    url = row["URL"]
    driver.get(url)

    class_name = "mw-parser-output"
    element_with_class = driver.find_element(By.CLASS_NAME, "mw-parser-output")

    elementos_p = element_with_class.find_elements(By.TAG_NAME, "p")

    if len(elementos_p) >= 3:
        tercer_p = elementos_p[2]

        lead_text = tercer_p.text
        turtle_secret_df.at[index, "Lead"] = lead_text
    else:
        print(f"No se encontraron suficientes elementos <p> para {row['Family']} en la URL {url}.\n")

driver.quit()


In [ ]:
turtle_secret_df